<a href="https://colab.research.google.com/github/Arjavjain100/Apple-Scab-Detection/blob/main/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt
from torchvision import datasets, transforms, models

In [ ]:

class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding=0))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        # print(x.shape)
        out = self.layer1(x)
        # print(out.shape)
        out = self.layer2(out)
        # print(out.shape)
        out = self.layer3(out)
        # print(out.shape)
        out = self.layer4(out)
        # print(out.shape)
        out = self.layer5(out)
        # print(out.shape)
        out = torch.flatten(out, start_dim =1)
        # print(out.shape)
        out = self.fc(out)
        # print(out.shape)
        out = self.fc1(out)
        # print(out.shape)
        out = self.fc2(out)
        # print(out.shape)
        return out

In [ ]:
def train(model, epochs, criterion, optimizer, trainloader, testloader, device):
  
  total_step = len(trainloader)

  for epoch in range(epochs):
      for i, (images, labels) in enumerate(trainloader):  
          # Move tensors to the configured device
          images = images.to(device)
          labels = labels.to(device)
          
          # Forward pass
          outputs = model(images)
          loss = criterion(outputs, labels)
          
          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                    .format(epoch+1, epochs, i+1, total_step, loss.item()))
              
      # Validation
      with torch.no_grad():
          correct = 0
          total = 0
          for images, labels in testloader:
              images = images.to(device)
              labels = labels.to(device)
              outputs = model(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()
              del images, labels, outputs
      
          print('Validation Accuracy: {} %'.format(100 * correct / total)) 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
model = AlexNet(2)
optimizer = opt.SGD(model.parameters(),lr=0.0005)
criterion = nn.CrossEntropyLoss()
model = model.to(device)
model

AlexNet(
  (layer1): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  

In [ ]:
batch_size = 16

transform = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((227, 227)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((227, 227)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])])
}
test_data = datasets.ImageFolder('/content/drive/MyDrive/Minor Project 1/Test', transform = transform['test'])
train_data = datasets.ImageFolder('/content/drive/MyDrive/Minor Project 1/Train',  transform = transform['train'])
train_loader = torch.utils.data.DataLoader(
    train_data, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(
    test_data, shuffle=True, batch_size=batch_size)


In [ ]:
train(model, 20 , criterion, optimizer, train_loader, test_loader, device)

Epoch [1/20], Step [17/17], Loss: 0.7557
Validation Accuracy: 32.432432432432435 %
Epoch [2/20], Step [17/17], Loss: 0.7891
Validation Accuracy: 32.432432432432435 %
Epoch [3/20], Step [17/17], Loss: 0.2939
Validation Accuracy: 32.432432432432435 %
Epoch [4/20], Step [17/17], Loss: 0.5650
Validation Accuracy: 32.432432432432435 %
Epoch [5/20], Step [17/17], Loss: 0.2256
Validation Accuracy: 32.432432432432435 %
Epoch [6/20], Step [17/17], Loss: 0.5747
Validation Accuracy: 32.432432432432435 %
Epoch [7/20], Step [17/17], Loss: 0.7961
Validation Accuracy: 29.72972972972973 %
Epoch [8/20], Step [17/17], Loss: 0.2744
Validation Accuracy: 32.432432432432435 %
Epoch [9/20], Step [17/17], Loss: 0.2909
Validation Accuracy: 32.432432432432435 %
Epoch [10/20], Step [17/17], Loss: 0.4082
Validation Accuracy: 32.432432432432435 %
Epoch [11/20], Step [17/17], Loss: 0.6274
Validation Accuracy: 32.432432432432435 %
Epoch [12/20], Step [17/17], Loss: 0.6891
Validation Accuracy: 32.432432432432435 %
Ep

In [ ]:
alex = models.alexnet(pretrained=True)

net_add =  nn.Linear(1000, 2)
model_pt = nn.Sequential(alex, net_add)
model_pt = model_pt.to(device)

model_pt

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=False)
      (1):

In [ ]:
train(model_pt, 20 , criterion, optimizer, train_loader, test_loader, device)

Epoch [1/20], Step [17/17], Loss: 0.6887
Validation Accuracy: 48.648648648648646 %
Epoch [2/20], Step [17/17], Loss: 0.4573
Validation Accuracy: 51.351351351351354 %
Epoch [3/20], Step [17/17], Loss: 0.2495
Validation Accuracy: 51.351351351351354 %
Epoch [4/20], Step [17/17], Loss: 0.5322
Validation Accuracy: 51.351351351351354 %
Epoch [5/20], Step [17/17], Loss: 0.7023
Validation Accuracy: 51.351351351351354 %
Epoch [6/20], Step [17/17], Loss: 0.5898
Validation Accuracy: 45.945945945945944 %
Epoch [7/20], Step [17/17], Loss: 1.3618
Validation Accuracy: 54.054054054054056 %
Epoch [8/20], Step [17/17], Loss: 0.4103
Validation Accuracy: 45.945945945945944 %
Epoch [9/20], Step [17/17], Loss: 0.8956
Validation Accuracy: 51.351351351351354 %
Epoch [10/20], Step [17/17], Loss: 0.7749
Validation Accuracy: 45.945945945945944 %
Epoch [11/20], Step [17/17], Loss: 0.3696
Validation Accuracy: 62.16216216216216 %
Epoch [12/20], Step [17/17], Loss: 0.4301
Validation Accuracy: 56.75675675675676 %
Epo